In [1]:
import xlrd
import xlwings as xw
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
xw.App.DisplayAlerts = False

In [4]:
# The core data extraction function
# This reads the Bond Payments sheet from every investor report in the given subfolder of my Office directory
def extract_reports_new_layout(subfolder):
    reports_loc = '/Users/Alex/Library/Group Containers/UBF8T346G9.Office/BSABS-HE10_INVESTOR_REPORTS/' + subfolder + '/'
    reports_dir = os.listdir(reports_loc)
    reports_dir.sort(key=lambda z: z[-8:]+z[-14:-12])
    data_list = [x for x in range(3)]
    for i in range(3):
        filename = reports_loc + reports_dir[i]
        investor_rpt = xw.Book(filename)
        bond_pmts = investor_rpt.sheets['Payment Stmt Pg1']
        from_excel = bond_pmts.range('A13:J51').options(pd.DataFrame).value
        from_excel.index.name = 'Class'
        from_excel.rename(index={None:'Total'},inplace=True)
        #from_excel.drop('Class',inplace=True)
        from_excel['Date'] = filename[-14:-4]
        from_excel.reindex(['I-A-1','I-A-2','I-A-3','I-M-1','I-M-2','I-M-3','I-M-4','I-M-5','I-M-6', \
                           'I-M-7','I-M-8','I-M-9','I-CE','I-P','I-R-1','I-R-2','I-R-3','I-RX', \
                           'II-1A-1','II-1A-2','II-1A-3','II-2A','II-3A','II-M-1','II-M-2','II-M-3', \
                           'II-M-4','II-M-5','II-M-6','II-M-7','II-M-8','II-M-9','II-CE','II-P', \
                           'II-R-1','II-RX'])
        data_list[i] = from_excel
        investor_rpt.close()
    data_df = pd.concat(data_list)
    data_df.rename(columns={'Through': 'Pass-Through Rate','Beginning':'Beginning Certificate Balance', \
                        'Principal':'Principal Payment','Interest':'Interest Payment', \
                        'Original':'Original Face Value (1)', 'Ending':'Ending Certificate Balance'}, \
                        inplace=True) 
    data_df['Date'] = pd.to_datetime(data_df['Date'])
    data_df.set_index('Date',append=True,inplace=True)
    data_df = data_df.reorder_levels(['Date','Class'])
    return data_df

In [5]:
extract_reports('June 2013 Through March 2020')

Pass-Through Rate Original Face Value (1)  \
Date       Class                                               
2013-06-25 Class            Rate (1)                 Balance   
           I-A-1           0.0030328             1.47515e+08   
           I-A-2           0.0039328              6.7747e+07   
           I-A-3           0.0043328              1.1213e+07   
           I-M-1           0.0053328              1.7235e+07   
           I-M-2           0.0054328              1.9813e+07   
           I-M-3           0.0057328               6.121e+06   
           I-M-4           0.0068328               7.732e+06   
           I-M-5           0.0070328               5.799e+06   
           I-M-6           0.0077328               4.832e+06   
           I-M-7           0.0199328               4.832e+06   
           I-M-8           0.0269328               3.865e+06   
           I-M-9           0.0269328               5.315e+06   
           I-CE                  N/A             3.22154e+08   
           I-P                     0                     100   
           II-1A-1         0.0026328             1.76117e+08   
           II-1A-2         0.0036328             1.01081e+08   
           II-1A-3         0.0043328              2.0339e+07   
           II-2A           0.0033328             2.01892e+08   
           II-3A           0.0033328             1.32221e+08   
           II-M-1          0.0048328              4.3161e+07   
           II-M-2          0.0049328              2.9466e+07   
           II-M-3          0.0052328              1.6601e+07   
           II-M-4          0.0058328               1.494e+07   
           II-M-5          0.0060328               1.494e+07   
           II-M-6          0.0067328              1.2865e+07   
           II-M-7          0.0109328               1.245e+07   
           II-M-8          0.0179328               7.055e+06   
           II-M-9          0.0244328              1.1205e+07   
           II-CE                 N/A             8.30025e+08   
           II-P                    0                     100   
           I-R-1                   0                       0   
           I-R-2                   0                       0   
           I-R-3                   0                       0   
           I-RX                    0                       0   
           II-R-1                  0                       0   
           II-RX                   0                       0   
           Total             Totals:             1.09635e+09   

                   Beginning Certificate Balance Principal Payment  \
Date       Class                                                     
2013-06-25 Class                        Balance               Paid   
           I-A-1                          246223            246223   
           I-A-2                      6.7747e+07            397874   
           I-A-3                      1.1213e+07                 0   
           I-M-1                      1.7235e+07                 0   
           I-M-2                      1.9813e+07                 0   
           I-M-3                       6.121e+06                 0   
           I-M-4                          832749                 0   
           I-M-5                               0                 0   
           I-M-6                               0                 0   
           I-M-7                               0                 0   
           I-M-8                               0                 0   
           I-M-9                               0                 0   
           I-CE                      1.23208e+08                 0   
           I-P                               100                 0   
           II-1A-1                    1.1644e+07            117110   
           II-1A-2                   1.01081e+08                 0   
           II-1A-3                    2.0339e+07                 0   
           II-2A                     9.56781e+07       1.3

In [89]:
# RETRIEVE ALL DATA FOR ONE SECURITY
june_2013.xs("I-A-1",level=1)

,Pass-Through Rate,Original Face Value (1),Beginning Certificate Balance,Principal Payment,Interest Payment,Total,Realized Loss,Loss Amount,Ending Certificate Balance
Date,,,,,,,,,
2013-06-25,0.0030328,1.47515e+08,246223,246223,58.08,246281,0,0,0


In [91]:
# RETRIEVE DATA FOR ALL MONTHS IN WHICH SECURITY I-M-9 HAD A NONZERO PRINCIPAL PAYMENT
june_2013.xs('I-M-9',level=1)[june_2013.xs('I-M-9',level=1)['Principal Payment']!=0]

,Pass-Through Rate,Original Face Value (1),Beginning Certificate Balance,Principal Payment,Interest Payment,Total,Realized Loss,Loss Amount,Ending Certificate Balance
Date,,,,,,,,,


In [8]:
# RETRIEVE ALL DATA FOR A RANGE OF DATES
data_df.loc[pd.date_range('2008-01-01','2008-03-01')]

CUSIP  Original Face Value (1)   NaN  \
Date       Class                                              
2008-01-25 I-A-1   07389RAA4             1.475150e+08  None   
           I-A-2   07389RAB2             6.774700e+07  None   
           I-A-3   07389RAC0             1.121300e+07  None   
           I-M-1   07389RAD8             1.723500e+07  None   
           I-M-2   07389RAE6             1.981300e+07  None   
...                      ...                      ...   ...   
2008-02-25 II-CE   07389RBK1             8.300254e+08     N   
           II-P    07389RBN5             1.000000e+02  None   
           II-R-1  07389RBL9             0.000000e+00  None   
           II-RX   07389RBM7             0.000000e+00  None   
           Total        None             7.943331e+08  None   

                   Beginning Certificate Balance  Principal Payment  \
Date       Class                                                      
2008-01-25 I-A-1                    9.033663e+07         3268125.25   
           I-A-2                    6.774700e+07               0.00   
           I-A-3                    1.121300e+07               0.00   
           I-M-1                    1.723500e+07               0.00   
           I-M-2                    1.981300e+07               0.00   
...                                          ...                ...   
2008-02-25 II-CE                    7.005190e+08               0.00   
           II-P                     1.000000e+02               0.00   
           II-R-1                   0.000000e+00               0.00   
           II-RX                    0.000000e+00               0.00   
           Total                    6.648280e+08         5831772.69   

                   Principal Adjustment or Loss  Deferred Interest  \
Date       Class                                                     
2008-01-25 I-A-1                            0.0                0.0   
           I-A-2                            0.0                0.0   
           I-A-3                            0.0                0.0   
           I-M-1                            0.0                0.0   
           I-M-2                            0.0                0.0   
...                                         ...                ...   
2008-02-25 II-CE                            0.0                0.0   
           II-P                             0.0                0.0   
           II-R-1                           0.0                0.0   
           II-RX                            0.0                0.0   
           Total                            0.0                0.0   

                   Ending Certificate Balance  Interest Payment(2)  \
Date       Class                                                     
2008-01-25 I-A-1                 8.706851e+07            374520.62   
           I-A-2                 6.774700e+07            285948.80   
           I-A-3                 1.121300e+07             47701.97   
           I-M-1                 1.723500e+07             74756.81   
           I-M-2                 1.981300e+07             86104.00   
...                                       ...                  ...   
2008-02-25 II-CE                 6.946872e+08            556234.00   
           II-P                  1.000000e+02             49334.52   
           II-R-1                0.000000e+00                 0.00   
           II-RX                 0.000000e+00                 0.00   
           Total                 6.589962e+08           2675603.70   

                   Interest Adjustment Pass-Through Rate  
Date       Class                                          
2008-01-25 I-A-1                  0.00           0.04975  
           I-A-2                  0.00           0.05065  
           I-A-3                  0.00           0.05105  
           I-M-1                  0.00           0.05205  
           I-M-2                  0.00           0.05215  
...                                ...      